# WarpBubbleSim Quickstart

This notebook provides a quick introduction to WarpBubbleSim, demonstrating the core functionality for simulating and visualizing warp bubble spacetimes.

## Installation

```bash
pip install -e ".[notebooks]"
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Import WarpBubbleSim
from warpbubblesim.metrics import AlcubierreMetric, NatarioMetric, get_metric, list_metrics
from warpbubblesim.gr import compute_einstein, compute_energy_density, check_energy_conditions
from warpbubblesim.gr.geodesics import integrate_geodesic
from warpbubblesim.viz.fields2d import plot_energy_density, plot_field_2d, plot_multiple_fields

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

## 1. Available Metrics

WarpBubbleSim implements several warp drive metrics from the literature:

In [ ]:
# List all available metrics
print("Available metrics:")
for name in list_metrics():
    print(f"  - {name}")

## 2. Creating a Metric

Let's create the classic Alcubierre warp bubble:

In [ ]:
# Create an Alcubierre metric with velocity v=1 (in units of c)
metric = AlcubierreMetric(v0=1.0, R=1.0, sigma=8.0)

# Print metric info
info = metric.info()
print(f"Metric: {info['name']}")
print(f"Citation: {info['citation']}")
print(f"Parameters: {info['parameters']}")

## 3. Computing the Metric Tensor

The metric tensor $g_{\mu\nu}$ at any spacetime point:

In [ ]:
# Get metric tensor at the bubble center
g_center = metric.metric(t=0, x=0, y=0, z=0)
print("Metric at bubble center (should be Minkowski):")
print(g_center)

print("\nMetric in the wall region:")
g_wall = metric.metric(t=0, x=1.0, y=0.3, z=0)
print(g_wall)

## 4. Shape Function

The shape function $f(r_s)$ determines the bubble profile:

In [ ]:
r = np.linspace(0, 3, 200)
f = [metric.shape_function(ri) for ri in r]

plt.figure(figsize=(8, 5))
plt.plot(r, f, 'b-', linewidth=2)
plt.axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=metric.R, color='gray', linestyle='--', alpha=0.5, label=f'R = {metric.R}')
plt.xlabel('$r_s$ (distance from bubble center)')
plt.ylabel('$f(r_s)$')
plt.title('Alcubierre Shape Function')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 5. Energy Density

The Alcubierre warp drive requires **negative energy density** (exotic matter) in the bubble wall:

In [ ]:
# Visualize energy density
fig, ax = plot_energy_density(
    metric,
    x_range=(-3, 3),
    y_range=(-3, 3),
    nx=100, ny=100,
    t=0.0
)
plt.title('Energy Density (Red = Negative/Exotic)')
plt.show()

In [ ]:
# Check energy density at specific points
points = [
    (0, 0, 0, 0),      # Center
    (0, 1.0, 0.3, 0),  # Wall region
    (0, 5.0, 0, 0),    # Far away
]

print("Energy density at different locations:")
for t, x, y, z in points:
    rho = metric.eulerian_energy_density_analytic(t, x, y, z)
    print(f"  ({x}, {y}, {z}): rho = {rho:.4e}")

## 6. Energy Conditions

Classical energy conditions are violated by the Alcubierre metric:

In [ ]:
# Check energy conditions in the wall region
metric_func = metric.get_metric_func()
coords = np.array([0, 1.0, 0.3, 0])

conditions = check_energy_conditions(metric_func, coords)

print("Energy conditions in wall region:")
for name, (satisfied, value) in conditions.items():
    status = "SATISFIED" if satisfied else "VIOLATED"
    print(f"  {name}: {status} (value: {value:.2e})")

## 7. Geodesics

Integrate test particle trajectories through the warp bubble:

In [ ]:
# Integrate geodesics for particles at different starting positions
metric_func = metric.get_metric_func()

results = []
for x0 in [-3, -2, -1, 0, 1, 2, 3]:
    initial_coords = np.array([0.0, float(x0), 0.0, 0.0])
    initial_velocity = np.array([1.0, 0.0, 0.0, 0.0])  # Initially at rest
    
    result = integrate_geodesic(
        metric_func,
        initial_coords,
        initial_velocity,
        lambda_span=(0, 10),
        max_step=0.1
    )
    results.append(result)

# Plot geodesics
plt.figure(figsize=(10, 8))
for result in results:
    coords = result['coords']
    plt.plot(coords[:, 1], coords[:, 0], alpha=0.7)

# Draw bubble trajectory
t_range = np.linspace(0, 10, 100)
x_bubble = [metric.bubble_center(t) for t in t_range]
plt.plot(x_bubble, t_range, 'k--', linewidth=2, label='Bubble center')

plt.xlabel('x')
plt.ylabel('t')
plt.title('Geodesics in Alcubierre Spacetime')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 8. Comparing Metrics

Compare the Alcubierre and Natario metrics:

In [ ]:
alcubierre = AlcubierreMetric(v0=1.0)
natario = NatarioMetric(v0=1.0)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

for ax, m, title in [(axes[0], alcubierre, 'Alcubierre'),
                      (axes[1], natario, 'Natario')]:
    plot_energy_density(m, x_range=(-3, 3), y_range=(-3, 3), 
                        nx=80, ny=80, t=0.0, ax=ax)
    ax.set_title(f'{title} Energy Density')

plt.tight_layout()
plt.show()

## 9. Multiple Field Visualization

View shape function, energy density, and expansion scalar together:

In [ ]:
fig = plot_multiple_fields(
    alcubierre,
    fields=['shape', 'energy_density', 'expansion'],
    x_range=(-3, 3),
    y_range=(-3, 3),
    nx=80, ny=80,
    t=0.0
)
plt.tight_layout()
plt.show()

## 10. Using the Registry

You can also access metrics by name:

In [ ]:
# Get metric by name
metric = get_metric('alcubierre', v0=2.0, R=1.5)
print(f"Created: {metric.name} with v0={metric.v0}, R={metric.R}")

# Try another metric
metric = get_metric('bobrick_martire', v0=0.3, positive_energy=True)
print(f"Created: {metric.name} (subluminal, positive energy)")

## Next Steps

- **01_alcubierre.ipynb**: Deep dive into the Alcubierre metric
- **02_natario.ipynb**: Expansion-free Natario metric
- **03_vdbroek.ipynb**: Van Den Broeck pocket geometry
- **04_white_toroidal.ipynb**: Toroidal energy distribution
- **05_bobrick_martire.ipynb**: Physical warp drives
- **06_lentz.ipynb**: Soliton warp drives

## Conventions

- **Metric signature**: (-,+,+,+)
- **Index ordering**: (t,x,y,z) = (0,1,2,3)
- **Units**: G = c = 1 (geometric units)